# Food Vision Model
> Prediciting 101 different Classes

In [2]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2023-11-08 13:11:42--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py.1’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-11-08 13:11:42 (82.9 MB/s) - ‘helper_functions.py.1’ saved [10246/10246]



In [3]:
from helper_functions import *

In [4]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [5]:
data_sets_list = tfds.list_builders()

(data_sets_list[:5]),('food101' in data_sets_list)

(['abstract_reasoning', 'accentdb', 'aeslc', 'aflw2k3d', 'ag_news_subset'],
 True)

In [6]:
(train_data,test_data), ds_info = tfds.load(name='food101',
                                            split=['train','validation'],
                                            shuffle_files=True,
                                            as_supervised=True,
                                            with_info=True)

In [7]:
train_data

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, None, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [9]:
def preprocess_img(image, label, img_shape=224):
  image = tf.image.resize(image,[img_shape,img_shape])
  return tf.cast(image,tf.float32), label

In [10]:
train_data = train_data.map(map_func=preprocess_img,
                            num_parallel_calls= tf.data.AUTOTUNE)

train_data = train_data.shuffle(buffer_size=1000).batch(batch_size=32).prefetch(buffer_size=tf.data.AUTOTUNE)



test_data = test_data.map(map_func=preprocess_img,
                            num_parallel_calls= tf.data.AUTOTUNE)

test_data = test_data.shuffle(buffer_size=1000).batch(batch_size=32).prefetch(buffer_size=tf.data.AUTOTUNE)

In [9]:
train_data

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [14]:
check_point = 'model_checkpoints/cp.ckpt'
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    check_point,
    monitor='val_acc',
    save_best_only=True,
    save_weights_only=True,
    verbose=0
)

In [11]:
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')

In [16]:
from tensorflow.keras import layers
from tensorflow.keras import preprocessing

base_model = tf.keras.applications.EfficientNetB4(include_top=False)
base_model.trainable = False

inputs = layers.Input(shape=(224, 224, 3),
                      name = 'input_layer')

x = base_model(inputs,
               training=False
               )

x = layers.GlobalAveragePooling2D(name='avg_pooling')(x)

outputs = layers.Dense(101,
                       activation= 'softmax',
                       dtype=tf.float32)(x)

model = tf.keras.Model(inputs,outputs)

model.compile(
    loss= tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer= tf.keras.optimizers.Adam(),
    metrics=['accuracy']
    )

history = model.fit(
    train_data,
    epochs=3,
    steps_per_epoch=len(train_data),
    validation_data=test_data,
    validation_steps=int(0.15*len(test_data)),
    callbacks=[model_checkpoint,
               create_tensorboard_callback(
                  'FOOD_VISION_101',
                  'Efficient_net'
               )]
)

71686520/71686520 [==============================] - 0s 0us/step
Saving TensorBoard log files to: FOOD_VISION_101/Efficient_net/20231108-132153
Epoch 1/3
2368/2368 [==============================] - ETA: 0s - loss: 1.7852 - accuracy: 0.5575

2368/2368 [==============================] - 256s 102ms/step - loss: 1.7852 - accuracy: 0.5575 - val_loss: 1.2544 - val_accuracy: 0.6703
Epoch 2/3
2367/2368 [============================>.] - ETA: 0s - loss: 1.3076 - accuracy: 0.6581

2368/2368 [==============================] - 250s 104ms/step - loss: 1.3077 - accuracy: 0.6581 - val_loss: 1.1473 - val_accuracy: 0.6888
Epoch 3/3
2367/2368 [============================>.] - ETA: 0s - loss: 1.1618 - accuracy: 0.6937

2368/2368 [==============================] - 248s 104ms/step - loss: 1.1618 - accuracy: 0.6937 - val_loss: 1.1122 - val_accuracy: 0.7071


In [21]:
for layer in base_model.layers[-10:]:
  layer.trainable = True

In [18]:
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 efficientnetb4 (Functional  (None, None, None, 1792   17673823  
 )                           )                                   
                                                                 
 avg_pooling (GlobalAverage  (None, 1792)              0         
 Pooling2D)                                                      
                                                                 
 dense_3 (Dense)             (None, 101)               181093    
                                                                 
Total params: 17854916 (68.11 MB)
Trainable params: 181093 (707.39 KB)
Non-trainable params: 17673823 (67.42 MB)
_________________________________________________________________


In [19]:
for layer in base_model.layers:
  print(layer.name,layer.trainable)

input_4 False
rescaling_6 False
normalization_3 False
rescaling_7 False
stem_conv_pad False
stem_conv False
stem_bn False
stem_activation False
block1a_dwconv False
block1a_bn False
block1a_activation False
block1a_se_squeeze False
block1a_se_reshape False
block1a_se_reduce False
block1a_se_expand False
block1a_se_excite False
block1a_project_conv False
block1a_project_bn False
block1b_dwconv False
block1b_bn False
block1b_activation False
block1b_se_squeeze False
block1b_se_reshape False
block1b_se_reduce False
block1b_se_expand False
block1b_se_excite False
block1b_project_conv False
block1b_project_bn False
block1b_drop False
block1b_add False
block2a_expand_conv False
block2a_expand_bn False
block2a_expand_activation False
block2a_dwconv_pad False
block2a_dwconv False
block2a_bn False
block2a_activation False
block2a_se_squeeze False
block2a_se_reshape False
block2a_se_reduce False
block2a_se_expand False
block2a_se_excite False
block2a_project_conv False
block2a_project_bn False
b

In [22]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(lr=0.0001),
              metrics=['accuracy'])
history_fine = model.fit(
    train_data,
    epochs=6,
    steps_per_epoch=len(train_data),
    validation_data=test_data,
    validation_steps=int(0.12*len(test_data)),
    initial_epoch=3,
    callbacks=[model_checkpoint,
               create_tensorboard_callback(
                  'FOOD_VISION_101',
                  'Efficient_net_fine_tuned'
               )]
)

Saving TensorBoard log files to: FOOD_VISION_101/Efficient_net_fine_tuned/20231108-133601
Epoch 4/6
2367/2368 [============================>.] - ETA: 0s - loss: 1.0505 - accuracy: 0.7203

2368/2368 [==============================] - 255s 100ms/step - loss: 1.0505 - accuracy: 0.7203 - val_loss: 1.0909 - val_accuracy: 0.7035
Epoch 5/6
2367/2368 [============================>.] - ETA: 0s - loss: 0.9745 - accuracy: 0.7391

2368/2368 [==============================] - 238s 100ms/step - loss: 0.9745 - accuracy: 0.7392 - val_loss: 1.0899 - val_accuracy: 0.7015
Epoch 6/6
2367/2368 [============================>.] - ETA: 0s - loss: 0.9150 - accuracy: 0.7556

2368/2368 [==============================] - 243s 102ms/step - loss: 0.9150 - accuracy: 0.7556 - val_loss: 1.0789 - val_accuracy: 0.7084


In [ ]:
base_model.trainable = True
for layer in base_model.layers[:5]:
  print(layer.name,layer.trainable)

In [ ]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(lr=0.001),
              metrics=['accuracy'])
history_fine_all = model.fit(
    train_data,
    epochs=101,
    steps_per_epoch=len(train_data),
    validation_data=test_data,
    validation_steps=int(0.12*len(test_data)),
    callbacks=[model_checkpoint,
               create_tensorboard_callback(
                  'FOOD_VISION_101',
                  'Efficient_net_fine_tuned'
               )]
)

In [12]:
from tensorflow.keras import layers
from tensorflow.keras import preprocessing


base_model = tf.keras.applications.EfficientNetB0(include_top=False)
base_model.trainable = True

inputs = layers.Input(shape=(224, 224, 3),
                      name = 'input_layer')

x = base_model(inputs)

x = layers.GlobalAveragePooling2D(name='avg_pooling')(x)

outputs = layers.Dense(101,
                       activation= 'softmax',
                       dtype=tf.float32)(x)

model = tf.keras.Model(inputs,outputs)

model.compile(
    loss= tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer= tf.keras.optimizers.Adam(),
    metrics=['accuracy']
    )

history = model.fit(
    train_data,
    epochs=101,
    steps_per_epoch=len(train_data),
    validation_data=test_data,
    validation_steps=int(0.12*len(test_data)),
)

Epoch 1/101
2368/2368 [==============================] - 459s 173ms/step - loss: 1.6598 - accuracy: 0.5766 - val_loss: 1.0612 - val_accuracy: 0.7074
Epoch 2/101
  60/2368 [..............................] - ETA: 6:37 - loss: 1.1799 - accuracy: 0.6979

KeyboardInterrupt: ignored